# 배송이 지연된 주문의 평균 지연일

In [ ]:
### 배송이 지연된 주문의 평균 지연일 ###

# 주문 데이터 불러오기
orders = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_orders_dataset.csv")

# 날짜 컬럼 변환
orders["order_estimated_delivery_date"] = pd.to_datetime(orders["order_estimated_delivery_date"])
orders["order_delivered_customer_date"] = pd.to_datetime(orders["order_delivered_customer_date"])

# 유효한 날짜가 있는 주문만 필터링
delivered_orders = orders.dropna(subset=["order_estimated_delivery_date", "order_delivered_customer_date"])

# 배송 지연일 계산 (예상일 - 실제 도착일)
delivered_orders["delivery_delay"] = (
    delivered_orders["order_delivered_customer_date"] - delivered_orders["order_estimated_delivery_date"]
).dt.days

# 지연된 주문만 필터링 (delay > 0인 경우만)
delayed_orders = delivered_orders[delivered_orders["delivery_delay"] > 0]

# 평균 지연일 계산
average_delay = delayed_orders["delivery_delay"].mean()

print(f"배송이 지연된 주문의 평균 지연일: {average_delay:.2f}일")

배송이 지연된 주문의 평균 지연일: 10.62일


/tmp/ipython-input-9-1214798611.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delivered_orders["delivery_delay"] = (


# 각 주마다의 주문갯수, 배송지연 갯수

In [ ]:
### 각 주마다의 주문갯수와 배송지연 갯수 ###

# CSV 파일 불러오기
orders_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_orders_dataset.csv")
customers_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_customers_dataset.csv")

# 날짜 처리
orders_df["order_estimated_delivery_date"] = pd.to_datetime(orders_df["order_estimated_delivery_date"])
orders_df["order_delivered_customer_date"] = pd.to_datetime(orders_df["order_delivered_customer_date"])

# 유효한 배송 정보만 필터링
delivered_orders = orders_df.dropna(subset=["order_estimated_delivery_date", "order_delivered_customer_date"]).copy()

# 배송 지연일 계산 및 지연 여부
delivered_orders["delivery_delay"] = (
    delivered_orders["order_delivered_customer_date"] - delivered_orders["order_estimated_delivery_date"]
).dt.days
delivered_orders["is_delayed"] = delivered_orders["delivery_delay"] > 0

# 고객 정보 병합 → 주(state) 정보 확보
merged_df = pd.merge(delivered_orders, customers_df, on="customer_id", how="left")

# 주(state) 단위로 그룹화
state_grouped = merged_df.groupby("customer_state").agg(
    total_orders=("order_id", "count"),
    delayed_orders=("is_delayed", "sum")
).reset_index()

# 지연률(%) 계산
state_grouped["delay_rate(%)"] = (state_grouped["delayed_orders"] / state_grouped["total_orders"]) * 100

# 주문 많은 순으로 정렬
state_grouped = state_grouped.sort_values(by="total_orders", ascending=False)

# 결과 출력
print(state_grouped)

   customer_state  total_orders  delayed_orders  delay_rate(%)
25             SP         40495            1820       4.494382
18             RJ         12353            1495      12.102323
10             MG         11355             520       4.579480
22             RS          5344             325       6.081587
17             PR          4923             199       4.042251
23             SC          3547             291       8.204116
4              BA          3256             396      12.162162
6              DF          2080             118       5.673077
7              ES          1995             214      10.726817
8              GO          1957             128       6.540623
15             PE          1593             153       9.604520
5              CE          1279             176      13.760751
13             PA           946             106      11.205074
12             MT           886              53       5.981941
9              MA           717             125      17

# 각 주의 배송지연률 계산

In [ ]:
### 주 기준 배송 지연률계산 ###

import pandas as pd
### 주 기준 배송 지연률계산 ###
# 데이터 불러오기
orders_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_orders_dataset.csv")
customers_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_customers_dataset.csv")
# 날짜 처리
orders_df["order_estimated_delivery_date"] = pd.to_datetime(orders_df["order_estimated_delivery_date"])
orders_df["order_delivered_customer_date"] = pd.to_datetime(orders_df["order_delivered_customer_date"])
# 유효한 배송만 필터링
delivered_orders = orders_df.dropna(subset=["order_estimated_delivery_date", "order_delivered_customer_date"]).copy()
# 배송 지연일 및 지연 여부 계산
delivered_orders["delivery_delay"] = (
    delivered_orders["order_delivered_customer_date"] - delivered_orders["order_estimated_delivery_date"]
).dt.days
delivered_orders["is_delayed"] = delivered_orders["delivery_delay"] > 0
# 고객 정보 병합
merged_df = pd.merge(delivered_orders, customers_df, on="customer_id", how="left")
# 주(state) 단위 그룹화
state_grouped = merged_df.groupby("customer_state").agg(
    total_orders=("order_id", "count"),
    delayed_orders=("is_delayed", "sum")
).reset_index()
# 지연률 계산
state_grouped["delay_rate(%)"] = (state_grouped["delayed_orders"] / state_grouped["total_orders"]) * 100
# ✅ 지연률 높은 순으로 정렬
state_grouped = state_grouped.sort_values(by="delay_rate(%)", ascending=False)
# 결과 출력
print(state_grouped)

   customer_state  total_orders  delayed_orders  delay_rate(%)
1              AL           397              85      21.410579
9              MA           717             125      17.433752
24             SE           335              51      15.223881
16             PI           476              66      13.865546
5              CE          1279             176      13.760751
21             RR            41               5      12.195122
4              BA          3256             396      12.162162
18             RJ         12353            1495      12.102323
13             PA           946             106      11.205074
7              ES          1995             214      10.726817
14             PB           517              54      10.444874
26             TO           274              27       9.854015
11             MS           701              68       9.700428
15             PE          1593             153       9.604520
19             RN           474              44       9

In [ ]:
### 주(customer_state)이름 (배송지연률 기준)
## 상위 5개 주
# AL = Alagoas
# MA = Maranhão
# SE = Sergipe
# PI = Piauí
# CE = Ceará

## 하위 5개 주
# AM = Amazonas
# RO = Rondônia
# AP = Amapá
# AC = Acre
# PR = Paraná

# 배송 지연률이 가장 높은 주 TOP5

In [ ]:
### 배송 지연률이 가장 높은 주 탑5 ###

orders_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_orders_dataset.csv")
customers_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_customers_dataset.csv")

# 날짜 처리
orders_df["order_estimated_delivery_date"] = pd.to_datetime(orders_df["order_estimated_delivery_date"])
orders_df["order_delivered_customer_date"] = pd.to_datetime(orders_df["order_delivered_customer_date"])

# 유효한 배송만 필터링
delivered_orders = orders_df.dropna(subset=["order_estimated_delivery_date", "order_delivered_customer_date"]).copy()

# 배송 지연일 및 지연 여부 계산
delivered_orders["delivery_delay"] = (
    delivered_orders["order_delivered_customer_date"] - delivered_orders["order_estimated_delivery_date"]
).dt.days
delivered_orders["is_delayed"] = delivered_orders["delivery_delay"] > 0

delivered_orders.loc[delivered_orders["is_delayed"] == True]

# 고객 정보 병합
merged_df = pd.merge(delivered_orders, customers_df, on="customer_id", how="left")

#이전 주별 계산까지와 달라지는 부분
city_grouped = merged_df.groupby(["customer_state", "customer_city"]).agg(
    total_orders=("order_id", "count"),
    delayed_orders=("is_delayed", "sum")
).reset_index()
city_grouped["delay_rate"] = city_grouped["delayed_orders"] / city_grouped["total_orders"] #.001/10 removed the extra calculations here
city_grouped_AL = city_grouped[city_grouped["customer_state"] == "AL"]
city_grouped_AL_sorted = city_grouped_AL.sort_values(by='delay_rate', ascending=False).reset_index(drop=True)

city_grouped_AL_sorted.head(60)

## 도시 갯수: 66개
## 배송이 가장 많은 도시: maceio

,customer_state,customer_city,total_orders,delayed_orders,delay_rate
0,AL,igaci,2,2,1.000000
1,AL,lagoa da canoa,1,1,1.000000
2,AL,monteiropolis,1,1,1.000000
3,AL,maravilha,2,1,0.500000
4,AL,sao miguel dos campos,2,1,0.500000
5,AL,campo alegre,2,1,0.500000
6,AL,igreja nova,2,1,0.500000
7,AL,inhapi,2,1,0.500000
8,AL,sao luis do quitunde,2,1,0.500000
9,AL,maragogi,5,2,0.400000


In [ ]:
import pandas as pd

orders_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_orders_dataset.csv")
customers_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_customers_dataset.csv")

# 날짜 처리
orders_df["order_estimated_delivery_date"] = pd.to_datetime(orders_df["order_estimated_delivery_date"])
orders_df["order_delivered_customer_date"] = pd.to_datetime(orders_df["order_delivered_customer_date"])

# 유효한 배송만 필터링
delivered_orders = orders_df.dropna(subset=["order_estimated_delivery_date", "order_delivered_customer_date"]).copy()

# 배송 지연일 및 지연 여부 계산
delivered_orders["delivery_delay"] = (
    delivered_orders["order_delivered_customer_date"] - delivered_orders["order_estimated_delivery_date"]
).dt.days
delivered_orders["is_delayed"] = delivered_orders["delivery_delay"] > 0

delivered_orders.loc[delivered_orders["is_delayed"] == True]

# 고객 정보 병합
merged_df = pd.merge(delivered_orders, customers_df, on="customer_id", how="left")

#이전 주별 계산까지와 달라지는 부분
city_grouped = merged_df.groupby(["customer_state", "customer_city"]).agg(
    total_orders=("order_id", "count"),
    delayed_orders=("is_delayed", "sum")
).reset_index()
city_grouped["delay_rate"] = city_grouped["delayed_orders"] / city_grouped["total_orders"] #.001/10 removed the extra calculations here
city_grouped_MA = city_grouped[city_grouped["customer_state"] == "MA"]
city_grouped_MA_sorted = city_grouped_MA.sort_values(by='delay_rate', ascending=False).reset_index(drop=True)

city_grouped_MA_sorted.head(60)

## 도시 갯수: 120개
## 배송이 가장 많은 도시: sao luis

,customer_state,customer_city,total_orders,delayed_orders,delay_rate
0,MA,bela vista do maranhao,1,1,1.000000
1,MA,formosa da serra negra,1,1,1.000000
2,MA,capinzal do norte,2,2,1.000000
3,MA,candido mendes,1,1,1.000000
4,MA,goncalves dias,1,1,1.000000
5,MA,sao joao do paraiso,1,1,1.000000
6,MA,maioba,2,2,1.000000
7,MA,joselandia,1,1,1.000000
8,MA,santa quiteria do maranhao,1,1,1.000000
9,MA,sao joao batista,1,1,1.000000


In [ ]:
import pandas as pd

orders_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_orders_dataset.csv")
customers_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_customers_dataset.csv")

# 날짜 처리
orders_df["order_estimated_delivery_date"] = pd.to_datetime(orders_df["order_estimated_delivery_date"])
orders_df["order_delivered_customer_date"] = pd.to_datetime(orders_df["order_delivered_customer_date"])

# 유효한 배송만 필터링
delivered_orders = orders_df.dropna(subset=["order_estimated_delivery_date", "order_delivered_customer_date"]).copy()

# 배송 지연일 및 지연 여부 계산
delivered_orders["delivery_delay"] = (
    delivered_orders["order_delivered_customer_date"] - delivered_orders["order_estimated_delivery_date"]
).dt.days
delivered_orders["is_delayed"] = delivered_orders["delivery_delay"] > 0

delivered_orders.loc[delivered_orders["is_delayed"] == True]

# 고객 정보 병합
merged_df = pd.merge(delivered_orders, customers_df, on="customer_id", how="left")

#이전 주별 계산까지와 달라지는 부분
city_grouped = merged_df.groupby(["customer_state", "customer_city"]).agg(
    total_orders=("order_id", "count"),
    delayed_orders=("is_delayed", "sum")
).reset_index()
city_grouped["delay_rate"] = city_grouped["delayed_orders"] / city_grouped["total_orders"] #.001/10 removed the extra calculations here
city_grouped_SE = city_grouped[city_grouped["customer_state"] == "SE"]
city_grouped_SE_sorted = city_grouped_SE.sort_values(by='delay_rate', ascending=False).reset_index(drop=True)

city_grouped_SE_sorted.head(60)

## 도시 갯수: 45개
## 배송이 가장 많은 도시: aracaju

,customer_state,customer_city,total_orders,delayed_orders,delay_rate
0,SE,indiaroba,1,1,1.000000
1,SE,pinhao,1,1,1.000000
2,SE,macambira,1,1,1.000000
3,SE,itaporanga d'ajuda,1,1,1.000000
4,SE,barra dos coqueiros,3,2,0.666667
5,SE,carmopolis,2,1,0.500000
6,SE,itabaianinha,2,1,0.500000
7,SE,umbauba,3,1,0.333333
8,SE,capela,3,1,0.333333
9,SE,cedro de sao joao,3,1,0.333333


In [ ]:
import pandas as pd

orders_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_orders_dataset.csv")
customers_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_customers_dataset.csv")

# 날짜 처리
orders_df["order_estimated_delivery_date"] = pd.to_datetime(orders_df["order_estimated_delivery_date"])
orders_df["order_delivered_customer_date"] = pd.to_datetime(orders_df["order_delivered_customer_date"])

# 유효한 배송만 필터링
delivered_orders = orders_df.dropna(subset=["order_estimated_delivery_date", "order_delivered_customer_date"]).copy()

# 배송 지연일 및 지연 여부 계산
delivered_orders["delivery_delay"] = (
    delivered_orders["order_delivered_customer_date"] - delivered_orders["order_estimated_delivery_date"]
).dt.days
delivered_orders["is_delayed"] = delivered_orders["delivery_delay"] > 0

delivered_orders.loc[delivered_orders["is_delayed"] == True]

# 고객 정보 병합
merged_df = pd.merge(delivered_orders, customers_df, on="customer_id", how="left")

#이전 주별 계산까지와 달라지는 부분
city_grouped = merged_df.groupby(["customer_state", "customer_city"]).agg(
    total_orders=("order_id", "count"),
    delayed_orders=("is_delayed", "sum")
).reset_index()
city_grouped["delay_rate"] = city_grouped["delayed_orders"] / city_grouped["total_orders"] #.001/10 removed the extra calculations here
city_grouped_PI = city_grouped[city_grouped["customer_state"] == "PI"]
city_grouped_PI_sorted = city_grouped_PI.sort_values(by='delay_rate', ascending=False).reset_index(drop=True)

city_grouped_PI_sorted.head(60)

## 도시 갯수: 72개
## 배송이 가장 많은 도시: teresina

,customer_state,customer_city,total_orders,delayed_orders,delay_rate
0,PI,gilbues,1,1,1.000000
1,PI,pio ix,1,1,1.000000
2,PI,inhuma,1,1,1.000000
3,PI,barras,3,2,0.666667
4,PI,corrente,4,2,0.500000
5,PI,luzilandia,3,1,0.333333
6,PI,floriano,18,4,0.222222
7,PI,teresina,271,51,0.188192
8,PI,campo maior,7,1,0.142857
9,PI,picos,17,2,0.117647


In [ ]:
import pandas as pd

orders_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_orders_dataset.csv")
customers_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_customers_dataset.csv")

# 날짜 처리
orders_df["order_estimated_delivery_date"] = pd.to_datetime(orders_df["order_estimated_delivery_date"])
orders_df["order_delivered_customer_date"] = pd.to_datetime(orders_df["order_delivered_customer_date"])

# 유효한 배송만 필터링
delivered_orders = orders_df.dropna(subset=["order_estimated_delivery_date", "order_delivered_customer_date"]).copy()

# 배송 지연일 및 지연 여부 계산
delivered_orders["delivery_delay"] = (
    delivered_orders["order_delivered_customer_date"] - delivered_orders["order_estimated_delivery_date"]
).dt.days
delivered_orders["is_delayed"] = delivered_orders["delivery_delay"] > 0

delivered_orders.loc[delivered_orders["is_delayed"] == True]

# 고객 정보 병합
merged_df = pd.merge(delivered_orders, customers_df, on="customer_id", how="left")

#이전 주별 계산까지와 달라지는 부분
city_grouped = merged_df.groupby(["customer_state", "customer_city"]).agg(
    total_orders=("order_id", "count"),
    delayed_orders=("is_delayed", "sum")
).reset_index()
city_grouped["delay_rate"] = city_grouped["delayed_orders"] / city_grouped["total_orders"] #.001/10 removed the extra calculations here
city_grouped_CE = city_grouped[city_grouped["customer_state"] == "CE"]
city_grouped_CE_sorted = city_grouped_CE.sort_values(by='delay_rate', ascending=False).reset_index(drop=True)

city_grouped_CE_sorted.head(60)

## 도시 갯수: 161개
## 배송이 가장 많은 도시: fortaleza

,customer_state,customer_city,total_orders,delayed_orders,delay_rate
0,CE,jucas,1,1,1.000000
1,CE,vicosa do ceara,1,1,1.000000
2,CE,tabuleiro do norte,1,1,1.000000
3,CE,tururu,1,1,1.000000
4,CE,acarau,8,5,0.625000
5,CE,abaiara,2,1,0.500000
6,CE,arneiroz,2,1,0.500000
7,CE,frecheirinha,2,1,0.500000
8,CE,ocara,2,1,0.500000
9,CE,iracema,2,1,0.500000


# 주문량이 가장 많은 주 TOP3

In [ ]:
import pandas as pd

### 주문량이 가장 많은 주 TOP3 ###

orders_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_orders_dataset.csv")
customers_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_customers_dataset.csv")

# 날짜 처리
orders_df["order_estimated_delivery_date"] = pd.to_datetime(orders_df["order_estimated_delivery_date"])
orders_df["order_delivered_customer_date"] = pd.to_datetime(orders_df["order_delivered_customer_date"])

# 유효한 배송만 필터링
delivered_orders = orders_df.dropna(subset=["order_estimated_delivery_date", "order_delivered_customer_date"]).copy()

# 배송 지연일 및 지연 여부 계산
delivered_orders["delivery_delay"] = (
    delivered_orders["order_delivered_customer_date"] - delivered_orders["order_estimated_delivery_date"]
).dt.days
delivered_orders["is_delayed"] = delivered_orders["delivery_delay"] > 0

delivered_orders.loc[delivered_orders["is_delayed"] == True]

# 고객 정보 병합
merged_df = pd.merge(delivered_orders, customers_df, on="customer_id", how="left")

#이전 주별 계산까지와 달라지는 부분
city_grouped = merged_df.groupby(["customer_state", "customer_city"]).agg(
    total_orders=("order_id", "count"),
    delayed_orders=("is_delayed", "sum")
).reset_index()
city_grouped["delay_rate"] = city_grouped["delayed_orders"] / city_grouped["total_orders"] #.001/10 removed the extra calculations here
city_grouped_SP = city_grouped[city_grouped["customer_state"] == "SP"]
city_grouped_SP_sorted = city_grouped_SP.sort_values(by='delay_rate', ascending=False).reset_index(drop=True)

city_grouped_SP_sorted.head(60)

## 도시 갯수 627개
## 배송이 가장 많은 도시: sao paulo

,customer_state,customer_city,total_orders,delayed_orders,delay_rate
0,SP,aramina,1,1,1.000000
1,SP,serra azul,1,1,1.000000
2,SP,rubineia,1,1,1.000000
3,SP,nova luzitania,1,1,1.000000
4,SP,ajapi,1,1,1.000000
5,SP,sarapui,2,1,0.500000
6,SP,nipoa,2,1,0.500000
7,SP,ribeirao branco,2,1,0.500000
8,SP,glicerio,2,1,0.500000
9,SP,rechan,2,1,0.500000


In [ ]:
import pandas as pd

orders_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_orders_dataset.csv")
customers_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_customers_dataset.csv")

# 날짜 처리
orders_df["order_estimated_delivery_date"] = pd.to_datetime(orders_df["order_estimated_delivery_date"])
orders_df["order_delivered_customer_date"] = pd.to_datetime(orders_df["order_delivered_customer_date"])

# 유효한 배송만 필터링
delivered_orders = orders_df.dropna(subset=["order_estimated_delivery_date", "order_delivered_customer_date"]).copy()

# 배송 지연일 및 지연 여부 계산
delivered_orders["delivery_delay"] = (
    delivered_orders["order_delivered_customer_date"] - delivered_orders["order_estimated_delivery_date"]
).dt.days
delivered_orders["is_delayed"] = delivered_orders["delivery_delay"] > 0

delivered_orders.loc[delivered_orders["is_delayed"] == True]

# 고객 정보 병합
merged_df = pd.merge(delivered_orders, customers_df, on="customer_id", how="left")

#이전 주별 계산까지와 달라지는 부분
city_grouped = merged_df.groupby(["customer_state", "customer_city"]).agg(
    total_orders=("order_id", "count"),
    delayed_orders=("is_delayed", "sum")
).reset_index()
city_grouped["delay_rate"] = city_grouped["delayed_orders"] / city_grouped["total_orders"] #.001/10 removed the extra calculations here
city_grouped_RJ = city_grouped[city_grouped["customer_state"] == "RJ"]
city_grouped_RJ_sorted = city_grouped_RJ.sort_values(by='delay_rate', ascending=False).reset_index(drop=True)

city_grouped_RJ_sorted.head(500)
## 도시 갯수: 145개
## 배송이 가장 많은 도시: rio de janeiro

,customer_state,customer_city,total_orders,delayed_orders,delay_rate
0,RJ,boa esperanca,1,1,1.0
1,RJ,portela,1,1,1.0
2,RJ,raposo,3,3,1.0
3,RJ,santa maria,1,1,1.0
4,RJ,sao sebastiao de campos,1,1,1.0
...,...,...,...,...,...
140,RJ,tocos,2,0,0.0
141,RJ,tres irmaos,1,0,0.0
142,RJ,travessao,4,0,0.0
143,RJ,tangua,6,0,0.0


In [ ]:
import pandas as pd

orders_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_orders_dataset.csv")
customers_df = pd.read_csv("/content/drive/MyDrive/20250702 프로젝트/olist_customers_dataset.csv")

# 날짜 처리
orders_df["order_estimated_delivery_date"] = pd.to_datetime(orders_df["order_estimated_delivery_date"])
orders_df["order_delivered_customer_date"] = pd.to_datetime(orders_df["order_delivered_customer_date"])

# 유효한 배송만 필터링
delivered_orders = orders_df.dropna(subset=["order_estimated_delivery_date", "order_delivered_customer_date"]).copy()

# 배송 지연일 및 지연 여부 계산
delivered_orders["delivery_delay"] = (
    delivered_orders["order_delivered_customer_date"] - delivered_orders["order_estimated_delivery_date"]
).dt.days
delivered_orders["is_delayed"] = delivered_orders["delivery_delay"] > 0

delivered_orders.loc[delivered_orders["is_delayed"] == True]

# 고객 정보 병합
merged_df = pd.merge(delivered_orders, customers_df, on="customer_id", how="left")

#이전 주별 계산까지와 달라지는 부분
city_grouped = merged_df.groupby(["customer_state", "customer_city"]).agg(
    total_orders=("order_id", "count"),
    delayed_orders=("is_delayed", "sum")
).reset_index()
city_grouped["delay_rate"] = city_grouped["delayed_orders"] / city_grouped["total_orders"] #.001/10 removed the extra calculations here
city_grouped_MG = city_grouped[city_grouped["customer_state"] == "MG"]
city_grouped_MG_sorted = city_grouped_MG.sort_values(by='delay_rate', ascending=False).reset_index(drop=True)

city_grouped_MG_sorted.head(1000)
## 도시 갯수: 740개
## 배송이 가장 많은 도시: belo horizonte

,customer_state,customer_city,total_orders,delayed_orders,delay_rate
0,MG,vermelho novo,1,1,1.0
1,MG,vila pereira,1,1,1.0
2,MG,bertopolis,1,1,1.0
3,MG,sao mateus de minas,1,1,1.0
4,MG,brasilandia de minas,1,1,1.0
...,...,...,...,...,...
735,MG,jequitinhonha,11,0,0.0
736,MG,jequitiba,2,0,0.0
737,MG,jequitai,2,0,0.0
738,MG,jenipapo de minas,1,0,0.0


# 찾았던 자료들 정리+출처

📦 브라질 지역별 배송지연률 분석 요약
1. 배송주문건수와 배송지연률은 비례하지 않는다.
출처: 프로젝트 자료
2. 배송지연률이 높은 곳과 낮은 곳은 지리적으로 근접한 경우가 많다.
출처: https://ko.wikipedia.org/wiki/%EB%B8%8C%EB%9D%BC%EC%A7%88%EC%9D%98_%EC%A3%BC
3. 배송지연률 상위 5개 주의 공통점은
브라질 동북부(Nordeste) 지역에 위치
빈부격차가 크고, 낙후된 인프라
농업, 어업 중심의 경제 구조
출처: https://frutasdobrasil.org/en/blog/conheca-a-regiao-nordeste-do-brasil/
4. 배송지연률이 낮은 곳은 주문량 자체가 낮은 경우가 많다.
출처: 프로젝트 자료
5. 배송지연률이 낮은곳은 낮은 인구밀도와 인구 집중현상이 나타난다.
출처:https://www.geo-ref.net/ph/bra.htm,
https://pmc.ncbi.nlm.nih.gov/articles/PMC3488306/
6. 배송지연률이 낮은 지역 4곳은 북부(Amazon 지역)에 위치하며, 도로인프라가 열약하며, 강 또는 항공 기반의 배송 방식을 이용한다.
출처:https://www.researchgate.net/figure/,Brazil-showing-major-existing-and-planned-highways-in-the-nine-states-of-the-Legal-Amazon_fig3_283136197,
https://www.sciencedirect.com/science/article/abs/pii/S0143622806000488
7. Parana 주는 예외적으로 남부에 위치하며, 교통 인프라가 우수하다.
출처:
https://en.wikipedia.org/wiki/Paran%C3%A1_%28state%29,
https://www.ishpes.org/ishpes-congress-2025/transportation,
https://en.clickpetroleoegas.com.br/,Important-Brazilian-highway-could-have-time-on-the-road-reduced-by-20-with-simple-measure-afch/#google_vignette